# 🕸️ LangGraph - Agentes Avanzados
## Framework Moderno para Workflows Complejos

**LangGraph** es el framework más nuevo de LangChain (2024-2025) para crear:
- 🔄 **Workflows con estados**
- 🤖 **Multi-agent systems**
- 🔁 **Loops y ciclos**
- 🌳 **Grafos de decisión**
- 🎯 **Control fino** sobre el flujo

### ¿Por qué LangGraph?
- ✅ Reemplaza `AgentExecutor` (deprecado)
- ✅ Control total del flujo de ejecución
- ✅ Mejor para producción
- ✅ Debugging más fácil
- ✅ Persistencia de estado

### Contenido:
1. **Conceptos básicos de LangGraph**
2. **Agent simple con LangGraph**
3. **Multi-agent system**
4. **Human-in-the-loop**
5. **Proyecto completo: Sistema de ventas**

## 📦 Instalación

In [ ]:
!pip install langgraph langchain-openai langchain-core langchain-community python-dotenv -q

## ⚙️ Configuración

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv('OPENAI_API_KEY'):
    raise ValueError("⚠️ OPENAI_API_KEY no encontrada")

print("✅ Configuración lista")

---
# 🌟 PARTE 1: Conceptos Básicos de LangGraph

LangGraph modela workflows como **grafos dirigidos**:

```
┌─────────┐
│  START  │
└────┬────┘
     │
     ▼
┌─────────┐
│  Node 1 │  (función que modifica el state)
└────┬────┘
     │
     ▼
┌─────────┐
│  Node 2 │
└────┬────┘
     │
     ▼
┌─────────┐
│   END   │
└─────────┘
```

## 1.1 Grafo Básico - Hello World

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

# 1. Definir el State (qué datos pasan entre nodos)
class State(TypedDict):
    mensaje: str
    contador: int

# 2. Definir nodos (funciones que modifican el state)
def nodo_saludo(state: State) -> State:
    """Primer nodo: genera saludo"""
    print("  🔵 Ejecutando nodo_saludo")
    return {
        "mensaje": f"Hola! {state['mensaje']}",
        "contador": state["contador"] + 1
    }

def nodo_despedida(state: State) -> State:
    """Segundo nodo: agrega despedida"""
    print("  🟢 Ejecutando nodo_despedida")
    return {
        "mensaje": f"{state['mensaje']} ¡Adiós!",
        "contador": state["contador"] + 1
    }

# 3. Construir el grafo
workflow = StateGraph(State)

# Agregar nodos
workflow.add_node("saludo", nodo_saludo)
workflow.add_node("despedida", nodo_despedida)

# Definir flujo (edges)
workflow.add_edge(START, "saludo")
workflow.add_edge("saludo", "despedida")
workflow.add_edge("despedida", END)

# Compilar
app = workflow.compile()

# 4. Ejecutar
print("\n" + "="*60)
print("🚀 EJECUTANDO GRAFO")
print("="*60 + "\n")

resultado = app.invoke({
    "mensaje": "Bienvenido a LangGraph",
    "contador": 0
})

print("\n✅ RESULTADO FINAL:")
print(f"  Mensaje: {resultado['mensaje']}")
print(f"  Nodos ejecutados: {resultado['contador']}")

## 1.2 Grafo con Decisiones (Conditional Edges)

In [ ]:
from typing import Literal

class StateDecision(TypedDict):
    numero: int
    resultado: str

def verificar_numero(state: StateDecision) -> StateDecision:
    print(f"  🔵 Verificando: {state['numero']}")
    return state

def numero_par(state: StateDecision) -> StateDecision:
    print("  ✅ Es PAR")
    return {"numero": state["numero"], "resultado": "PAR"}

def numero_impar(state: StateDecision) -> StateDecision:
    print("  ⚠️ Es IMPAR")
    return {"numero": state["numero"], "resultado": "IMPAR"}

# Función de decisión
def decidir_ruta(state: StateDecision) -> Literal["par", "impar"]:
    """Decide qué camino tomar basándose en el state"""
    if state["numero"] % 2 == 0:
        return "par"
    else:
        return "impar"

# Construir grafo con decisión
workflow_decision = StateGraph(StateDecision)

workflow_decision.add_node("verificar", verificar_numero)
workflow_decision.add_node("par", numero_par)
workflow_decision.add_node("impar", numero_impar)

workflow_decision.add_edge(START, "verificar")

# Edge condicional
workflow_decision.add_conditional_edges(
    "verificar",
    decidir_ruta,
    {"par": "par", "impar": "impar"}
)

workflow_decision.add_edge("par", END)
workflow_decision.add_edge("impar", END)

app_decision = workflow_decision.compile()

# Probar
print("\n" + "="*60)
print("🔀 GRAFO CON DECISIONES")
print("="*60 + "\n")

for num in [4, 7]:
    print(f"\nProbando con {num}:")
    resultado = app_decision.invoke({"numero": num, "resultado": ""})
    print(f"  Resultado: {resultado['resultado']}")

---
# 🤖 PARTE 2: Agent con LangGraph

Reemplazo moderno de `create_react_agent`

## 2.1 Definir Tools

In [ ]:
from langchain_core.tools import tool
from datetime import datetime, timedelta

@tool
def calcular_precio(precio: float, dias: int) -> str:
    """Calcula el precio total con descuentos.
    
    Args:
        precio: Precio por día
        dias: Número de días
    """
    total = precio * dias
    
    if dias >= 30:
        desc = 0.20
    elif dias >= 14:
        desc = 0.15
    else:
        desc = 0
    
    final = total * (1 - desc)
    return f"Total: L{final:.2f} (desc: {desc*100}%)"

@tool
def verificar_stock(producto: str) -> str:
    """Verifica disponibilidad de un producto.
    
    Args:
        producto: Nombre del producto
    """
    stock = {
        "demoledor": 3,
        "rotomartillo": 5,
        "compactador": 0
    }
    
    for key, cant in stock.items():
        if key in producto.lower():
            if cant > 0:
                return f"✅ {producto}: {cant} unidades disponibles"
            else:
                return f"❌ {producto}: Agotado"
    
    return f"⚠️ Producto no encontrado: {producto}"

tools = [calcular_precio, verificar_stock]
print(f"✅ Tools creadas: {[t.name for t in tools]}")

## 2.2 Crear Agent con LangGraph

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Crear agent con LangGraph (NO langchain-classic)
agent_graph = create_react_agent(
    llm,
    tools,
    state_modifier="Eres un asistente de ventas profesional."
)

print("✅ Agent creado con LangGraph")
print("\nEste es el REEMPLAZO moderno de AgentExecutor")

## 2.3 Ejecutar el Agent

In [ ]:
print("\n" + "="*60)
print("🤖 AGENT CON LANGGRAPH")
print("="*60)

# Pregunta 1
print("\n👤 Usuario: ¿Tienen rotomartillos disponibles?")
resultado = agent_graph.invoke({
    "messages": [("user", "¿Tienen rotomartillos disponibles?")]
})

# Obtener última respuesta
ultima_msg = resultado["messages"][-1]
print(f"🤖 Agent: {ultima_msg.content}")

# Pregunta 2
print("\n👤 Usuario: ¿Cuánto cuesta rentar un demoledor por 20 días a L550/día?")
resultado = agent_graph.invoke({
    "messages": [("user", "¿Cuánto cuesta rentar un demoledor por 20 días a L550/día?")]
})

ultima_msg = resultado["messages"][-1]
print(f"🤖 Agent: {ultima_msg.content}")

## 2.4 Ver el Grafo del Agent

In [ ]:
# Ver estructura interna del agent
print("\n🕸️ ESTRUCTURA DEL AGENT:")
print("\nNodos:", list(agent_graph.nodes.keys()))
print("\n💡 LangGraph maneja automáticamente el loop del agent")
print("   - Llama al LLM")
print("   - Ejecuta tools")
print("   - Repite hasta tener respuesta final")

---
# 👥 PARTE 3: Multi-Agent System

Múltiples agentes especializados trabajando juntos

## 3.1 Definir Agentes Especializados

In [ ]:
from typing import Annotated
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

# Agente 1: Experto en productos
def agente_productos(state: MessagesState):
    """Experto en catálogo de productos"""
    print("  🛠️ Agente de Productos trabajando...")
    
    system_msg = SystemMessage(content="""Eres un experto en productos de construcción.
    Conoces todo el catálogo de CONCESA.
    Tu trabajo es recomendar productos según las necesidades del cliente.""")
    
    messages = [system_msg] + state["messages"]
    response = llm.invoke(messages)
    
    return {"messages": [response]}

# Agente 2: Experto en precios
def agente_precios(state: MessagesState):
    """Experto en cálculos de precios y descuentos"""
    print("  💰 Agente de Precios trabajando...")
    
    llm_con_tool = llm.bind_tools([calcular_precio])
    
    system_msg = SystemMessage(content="""Eres un experto en precios y descuentos.
    Calculas cotizaciones y aplicas descuentos según días de renta.""")
    
    messages = [system_msg] + state["messages"]
    response = llm_con_tool.invoke(messages)
    
    return {"messages": [response]}

# Agente 3: Coordinador
def agente_coordinador(state: MessagesState):
    """Decide qué agente debe manejar la consulta"""
    print("  🎯 Coordinador analizando...")
    
    ultima_msg = state["messages"][-1].content.lower()
    
    if "precio" in ultima_msg or "costo" in ultima_msg or "cuánto" in ultima_msg:
        return "precios"
    else:
        return "productos"

print("✅ Agentes especializados creados")

## 3.2 Construir Sistema Multi-Agent

In [ ]:
# Construir grafo multi-agent
workflow_multi = StateGraph(MessagesState)

# Agregar agentes
workflow_multi.add_node("productos", agente_productos)
workflow_multi.add_node("precios", agente_precios)

# Edge condicional desde START
workflow_multi.add_conditional_edges(
    START,
    agente_coordinador,
    {"productos": "productos", "precios": "precios"}
)

# Todos terminan en END
workflow_multi.add_edge("productos", END)
workflow_multi.add_edge("precios", END)

# Compilar
sistema_multi = workflow_multi.compile()

print("✅ Sistema multi-agent creado")

## 3.3 Probar Multi-Agent System

In [ ]:
print("\n" + "="*60)
print("👥 SISTEMA MULTI-AGENT")
print("="*60)

# Consulta 1: Productos
print("\n👤 Usuario: ¿Qué equipos tienen para demolición?")
resultado = sistema_multi.invoke({
    "messages": [HumanMessage(content="¿Qué equipos tienen para demolición?")]
})
print(f"\n🤖 {resultado['messages'][-1].content}")

# Consulta 2: Precios
print("\n" + "-"*60)
print("\n👤 Usuario: ¿Cuánto cuesta rentar por 15 días?")
resultado = sistema_multi.invoke({
    "messages": [HumanMessage(content="¿Cuánto cuesta rentar un rotomartillo a L750/día por 15 días?")]
})
print(f"\n🤖 {resultado['messages'][-1].content}")

print("\n💡 El coordinador decidió automáticamente qué agente usar")

---
# 🙋 PARTE 4: Human-in-the-Loop

Pausar ejecución para aprobación humana

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

class StateAprobacion(TypedDict):
    accion: str
    aprobado: bool
    resultado: str

def solicitar_aprobacion(state: StateAprobacion) -> StateAprobacion:
    print(f"\n⏸️  PAUSADO: Requiere aprobación para '{state['accion']}'")
    return state

def ejecutar_accion(state: StateAprobacion) -> StateAprobacion:
    if state["aprobado"]:
        print(f"  ✅ Ejecutando: {state['accion']}")
        return {**state, "resultado": "Acción completada"}
    else:
        print(f"  ❌ Acción rechazada")
        return {**state, "resultado": "Acción cancelada"}

def decidir_ejecucion(state: StateAprobacion) -> Literal["ejecutar", "end"]:
    return "ejecutar" if state["aprobado"] else "end"

# Construir grafo
workflow_aprobacion = StateGraph(StateAprobacion)
workflow_aprobacion.add_node("aprobacion", solicitar_aprobacion)
workflow_aprobacion.add_node("ejecutar", ejecutar_accion)

workflow_aprobacion.add_edge(START, "aprobacion")
workflow_aprobacion.add_conditional_edges(
    "aprobacion",
    decidir_ejecucion,
    {"ejecutar": "ejecutar", "end": END}
)
workflow_aprobacion.add_edge("ejecutar", END)

# Compilar con checkpointer (para persistencia)
checkpointer = MemorySaver()
app_aprobacion = workflow_aprobacion.compile(checkpointer=checkpointer)

print("\n" + "="*60)
print("🙋 HUMAN-IN-THE-LOOP")
print("="*60)

# Simular aprobación
config = {"configurable": {"thread_id": "1"}}

# Paso 1: Pausar en aprobación
resultado = app_aprobacion.invoke(
    {"accion": "Aplicar descuento de 20%", "aprobado": False, "resultado": ""},
    config
)

# Paso 2: Humano aprueba
print("\n👤 Humano aprueba la acción...")
resultado = app_aprobacion.invoke(
    {"accion": "Aplicar descuento de 20%", "aprobado": True, "resultado": ""},
    config
)

print(f"\n✅ Resultado: {resultado['resultado']}")

---
# 🎯 PARTE 5: Proyecto Completo - Sistema de Ventas

Combinando todo: Multi-agent + RAG + Memory

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

PDF_PATH = 'Documentos - PDF/Catalogo_Equipos_Construccion.pdf'

if os.path.exists(PDF_PATH):
    print("🏗️ CONSTRUYENDO SISTEMA COMPLETO...\n")
    
    # 1. Cargar RAG
    loader = PyPDFLoader(PDF_PATH)
    docs = loader.load()
    
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = splitter.split_documents(docs)
    
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    vectorstore = FAISS.from_documents(chunks, embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
    
    print(f"✅ RAG configurado: {len(chunks)} chunks\n")
    
    # 2. Crear tool de búsqueda
    @tool
    def buscar_catalogo(consulta: str) -> str:
        """Busca productos en el catálogo.
        
        Args:
            consulta: Descripción del producto a buscar
        """
        docs = retriever.invoke(consulta)
        return "\n\n".join([d.page_content for d in docs])
    
    # 3. Actualizar tools
    tools_completo = tools + [buscar_catalogo]
    
    # 4. Crear agent completo
    sistema_ventas = create_react_agent(
        llm,
        tools_completo,
        state_modifier="""Eres un agente de ventas experto de CONCESA.
        Ayudas a clientes a encontrar equipos de construcción.
        Usa el catálogo para buscar productos.
        Calcula precios con descuentos.
        Siempre eres profesional y servicial."""
    )
    
    print("✅ Sistema completo creado con:")
    print("   - RAG (búsqueda en catálogo)")
    print("   - Tools de precios y stock")
    print("   - LangGraph agent")
    print("   - Memory integrada\n")
    
    # 5. Demo
    print("="*60)
    print("🎬 DEMO - SISTEMA DE VENTAS COMPLETO")
    print("="*60)
    
    conversacion = [
        "Hola, necesito equipos para demoler concreto",
        "¿Cuál es el más potente?",
        "¿Cuánto cuesta rentarlo por 20 días?",
        "¿Está disponible?"
    ]
    
    state_conversacion = {"messages": []}
    
    for pregunta in conversacion:
        print(f"\n👤 Cliente: {pregunta}")
        
        state_conversacion["messages"].append(("user", pregunta))
        resultado = sistema_ventas.invoke(state_conversacion)
        
        respuesta = resultado["messages"][-1].content
        print(f"🤖 Agente: {respuesta}")
        
        state_conversacion = resultado
        print("-"*60)
    
    print("\n✨ El sistema usó:")
    print("   1. RAG para buscar en el catálogo")
    print("   2. Tool de cálculo de precios")
    print("   3. Tool de verificación de stock")
    print("   4. Memoria conversacional")
    
else:
    print("⚠️ No se encontró el PDF")

---
# 📚 Resumen de LangGraph

## ✅ Ventajas sobre langchain-classic:

1. **Control total del flujo**
   - Defines exactamente cómo fluye la información
   - Decisiones condicionales explícitas
   - Loops personalizados

2. **Debugging más fácil**
   - Ves cada paso del grafo
   - Puedes inspeccionar el state en cualquier punto
   - Logs claros

3. **Persistencia de estado**
   - Checkpointers para guardar progreso
   - Puedes pausar y resumir
   - Human-in-the-loop natural

4. **Multi-agent nativo**
   - Múltiples agentes especializados
   - Coordinación automática
   - Escalable

5. **Production-ready**
   - Mejor manejo de errores
   - Timeouts configurables
   - Streaming de respuestas

## 🆚 Comparación:

| Aspecto | langchain-classic | LangGraph |
|---------|------------------|----------|
| **API** | `create_react_agent` | `StateGraph` |
| **Control** | Limitado | Total |
| **Debugging** | Difícil | Fácil |
| **Multi-agent** | Complejo | Nativo |
| **Estado** | No persiste | Persiste |
| **Human-loop** | Manual | Integrado |
| **Futuro** | Deprecado | Activo |

## 🎯 Cuándo usar LangGraph:

✅ **Usa LangGraph cuando:**
- Necesitas workflows complejos
- Múltiples agentes
- Control fino del flujo
- Aplicaciones de producción
- Human-in-the-loop

⚠️ **Quédate con LCEL simple cuando:**
- Workflows lineales simples
- Prototipos rápidos
- Aprendiendo conceptos

## 📖 Recursos:
- [LangGraph Docs](https://langchain-ai.github.io/langgraph/)
- [LangGraph Tutorials](https://langchain-ai.github.io/langgraph/tutorials/)
- [Multi-Agent Examples](https://github.com/langchain-ai/langgraph/tree/main/examples)